# Разработка модели детекции наклеек: Faster R-CNN

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

## Загрузка предобученной модели Faster R-CNN

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

## Пользовательский классификатор

In [ ]:
num_classes = 2  # фон и наклейка
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

## Загрузка и предобработка данных

In [ ]:
import os
import torch
from PIL import Image
import json

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotations, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.annotations = list(sorted(os.listdir(os.path.join(root, "annotations"))))

    def __getitem__(self, idx):
        # Загрузка изображений и аннотаций
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        ann_path = os.path.join(self.root, "annotations", self.annotations[idx])

        img = Image.open(img_path).convert("RGB")

        # Загрузка файла аннотации
        with open(ann_path) as f:
            boxes = json.load(f)["boxes"]

        # Конвертация координат боксов в тензор Torch
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        num_objs = len(boxes)
        # Пометка класса наклейки как 1 (предполагая, что 0 - это фон)
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # Дополнительные трансформации для обучения
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## Обучение модели

In [ ]:
from engine import train_one_epoch, evaluate
import utils

def train_model(model, data_loader, optimizer, device, num_epochs=3):
    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        lr_scheduler.step()

        # оценка на каждом шаге
        evaluate(model, data_loader, device=device)


In [ ]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

# Инициализация датасета, DataLoader и модели
dataset = CustomDataset(root="path/to/images", annotations="path/to/annotations", transforms=get_transform(train=True))

# Определение размеров выборок
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Разделение датасета на обучающую и тестовую выборки
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Инициализация оптимизатора
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Запуск обучения
train_model(model, train_loader, optimizer, device)


## Сохранение модели

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')

# Модель распознавания текста с наклеек: Tesseract OCR
 Является одним из наиболее широко используемых инструментов OCR, поддерживает множество языков и имеет хорошую точность распознавания текста.

In [ ]:
! pip install pytesseract


In [ ]:
import pytesseract
from PIL import Image

# Функция для извлечения текста из области изображения
def extract_text_from_image(image, box):
    # Обрезка изображения по указанной области
    cropped_image = image.crop((box[0], box[1], box[2], box[3]))
    # Распознавание текста с помощью Tesseract OCR
    text = pytesseract.image_to_string(cropped_image, lang='eng')
    return text

## Извлечение текста из изображений исходя из полученной ранее разметки

In [ ]:
# Загрузка обученной модели
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2)
model.load_state_dict(torch.load('fasterrcnn_resnet50_fpn.pth'))

# Перевод модели в режим оценки
model.eval()

In [ ]:
from torchvision.transforms import functional as F

def prepare_image(image_path):
    # Загрузка изображения
    image = Image.open(image_path)
    image = image.convert("RGB")
    # Преобразование изображения в тензор
    image = F.to_tensor(image)
    return image

In [ ]:
@torch.no_grad()  # Отключение расчета градиентов
def get_predictions(model, data_loader):
    model.eval()  # Перевод модели в режим оценки
    predictions = []
    for images, _ in data_loader:
        images = [image.to(device) for image in images]
        outputs = model(images)
        predictions.extend(outputs)
    return predictions

# Получение предсказаний, используя test_loader
predictions = get_predictions(model, test_loader)


In [ ]:
# Извлечение текста из каждой области
texts = [extract_text_from_image(img, target) for img, target in predictions]